In [23]:
import numpy as np
import pandas as pd
import math
import xlrd

In [24]:
scrap_rate = 1.2

In [47]:
demand_to_raw_material = pd.ExcelFile("excels/Output Points and Raw Materials Needed Sample Product.xlsx")
demand_to_raw_material = demand_to_raw_material.parse(demand_to_raw_material.sheet_names[0])

demand = pd.ExcelFile("excels/demand.xlsx")
demand = demand.parse(demand.sheet_names[0])
# sort demand by deadline time
index = pd.to_datetime(demand["deadline"], format="%Y%m%d:%H:%M:%S.%f").sort_values().index
demand = demand.iloc[index, :]

inventory = pd.ExcelFile("excels/inventory.xlsx")
inventory = inventory.parse(inventory.sheet_names[0])
inventory = {inventory.iloc[index, 0]: inventory.iloc[index, 1] for index in range(inventory.shape[0])}

In [48]:
def output_date(date_list, ending, starting):
    day_num = int(starting/(starting-ending)*len(date_list))
    return date_list[day_num]

In [49]:
finished_good = []
semi_finished_good = []
semi_finished_to_raw_material = {}

temp = 0
raw_material = []
for index, class_num in enumerate(demand_to_raw_material.iloc[1:, 1]):
    if float(class_num) == 1:
        finished_good.append(demand_to_raw_material.iloc[index+1, 0])
    if float(class_num) == 2:
        semi_finished_good.append(temp)
        semi_finished_to_raw_material[temp] = raw_material
        temp = demand_to_raw_material.iloc[index+1, 0]
        raw_material = []
    if float(class_num) == 3:
        raw_material.append(demand_to_raw_material.iloc[index+1, 0])
semi_finished_good.append(temp)
semi_finished_to_raw_material[temp] = raw_material
del semi_finished_to_raw_material[0]

In [50]:
time_temp = pd.Timestamp.today().date()
raw_materials = list(inventory.keys())
shortage = {}
for index in range(demand.shape[0]):
    deadline = demand['deadline'][index]
    date_list = list(pd.date_range(start=time_temp, end=deadline.date()))
    quantity = demand['quantity'][index]
    if demand['product_type'][index] in finished_good[1:]:
        for raw_material in semi_finished_good:
            if raw_material in shortage:
                pass
            else:
                initial = inventory[raw_material]
                inventory[raw_material] -= quantity * scrap_rate
                if inventory[raw_material] < 0: date = output_date(date_list, inventory[raw_material], initial)
                shortage[raw_material] = date
    else:
        raw_materials = semi_finished_to_raw_material[demand['product_type'][index]]
        for raw_material in raw_materials:
            if raw_material in shortage:
                pass
            else:
                initial = inventory[raw_material]
                inventory[raw_material] -= quantity * scrap_rate
                if inventory[raw_material] < 0: date = output_date(date_list, inventory[raw_material], initial)
                shortage[raw_material] = date

In [51]:
shortage

{'J0 Submount': Timestamp('2019-06-03 00:00:00', freq='D'),
 'Preform': Timestamp('2019-06-03 00:00:00', freq='D'),
 'Thermistor(TM5)': Timestamp('2019-06-03 00:00:00', freq='D')}